In [1]:
import requests
import json, os, subprocess, datetime

root_home_automation_directory = 'N:\\HomeAutomation'
gcp_oauth_file_name = 'GCP_OAuth.json'
auth_code_file_name = 'Auth_Code.json'
device_access_file_name = 'Device_Access_Project_ID.json'

gcp_oauth_path = f'{root_home_automation_directory}\\{gcp_oauth_file_name}'
auth_code_path = f'{root_home_automation_directory}\\{auth_code_file_name}'
device_access_path = f'{root_home_automation_directory}\\{device_access_file_name}'

gcp_oauth_json = {}
with open(gcp_oauth_path) as f:
    gcp_oauth_json = json.load(f)
    
device_access_json = {}
with open(device_access_path) as f:
    device_access_json = json.load(f)

auth_code_json = None
if os.path.isfile(auth_code_path):
    with open(auth_code_path) as f:
        auth_code_json = json.load(f)

In [93]:
# Write and execute bat file to get auth code
# take the value of 'code' from the result url and store it in Auth_Code.json as {'code':'<TheCode>'}
url = f'https://nestservices.google.com/partnerconnections/{device_access_json["project-id"]}/auth?\
redirect_uri=https://www.google.com&\
access_type=offline&\
prompt=consent&\
client_id={gcp_oauth_json["web"]["client_id"]}&\
response_type=code&\
scope=https://www.googleapis.com/auth/sdm.service'

request_permission_file_name = 'request_permission.bat'

request_permission_path = f'{root_home_automation_directory}\\{request_permission_file_name}'

bat_file = open(request_permission_path, 'w') 
record = f'start "" "{url}"'
bat_file.write(record) 
bat_file.close() 

subprocess.call([request_permission_path])

0

In [94]:
# Get tokens, Only egt one chance
auth_code_json = None
if os.path.isfile(auth_code_path):
    with open(auth_code_path) as f:
        auth_code_json = json.load(f)

url = f'https://www.googleapis.com/oauth2/v4/token?client_id={gcp_oauth_json["web"]["client_id"]}&client_secret={gcp_oauth_json["web"]["client_secret"]}&code={auth_code_json["code"]}&grant_type=authorization_code&redirect_uri=https://www.google.com'
response = requests.post(url=url)

In [110]:
token_response = json.loads((response.text))
token_response['assigned'] = datetime.datetime.strptime(response.headers['Date'], '%a, %d %b %Y %H:%M:%S %Z').strftime('%a, %d %b %Y %H:%M:%S %Z')
token_response['expires'] = (datetime.datetime.strptime(response.headers['Date'], '%a, %d %b %Y %H:%M:%S %Z') + datetime.timedelta(seconds=token_response['expires_in'])).strftime('%a, %d %b %Y %H:%M:%S %Z')
#del my_dict['key']

tokens_file_name = 'token.json'
active_tokens_file_name = 'active_token.json'

tokens_path = f'{root_home_automation_directory}\\{tokens_file_name}'
active_tokens_path = f'{root_home_automation_directory}\\{active_tokens_file_name}'

token_file = open(tokens_path, 'w') 
record = json.dumps(token_response)
token_file.write(record) 
token_file.close() 

token_file = open(active_tokens_path, 'w') 
record = json.dumps(token_response)
token_file.write(record) 
token_file.close() 

In [2]:
# refresh
tokens_file_name = 'token.json'
active_tokens_file_name = 'active_token.json'

tokens_path = f'{root_home_automation_directory}\\{tokens_file_name}'
active_tokens_path = f'{root_home_automation_directory}\\{active_tokens_file_name}'

token_json = None
if os.path.isfile(tokens_path):
    with open(tokens_path) as f:
        token_json = json.load(f)

url = f'https://www.googleapis.com/oauth2/v4/token?client_id={gcp_oauth_json["web"]["client_id"]}&client_secret={gcp_oauth_json["web"]["client_secret"]}&refresh_token={token_json["refresh_token"]}&grant_type=refresh_token'
response = requests.post(url=url)

token_response = json.loads((response.text))
token_response['assigned'] = datetime.datetime.strptime(response.headers['Date'], '%a, %d %b %Y %H:%M:%S %Z').strftime('%a, %d %b %Y %H:%M:%S %Z')
token_response['expires'] = (datetime.datetime.strptime(response.headers['Date'], '%a, %d %b %Y %H:%M:%S %Z') + datetime.timedelta(seconds=token_response['expires_in'])).strftime('%a, %d %b %Y %H:%M:%S %Z')

token_file = open(active_tokens_path, 'w') 
record = json.dumps(token_response)
token_file.write(record) 
token_file.close() 